In [ ]:
# ────────────────────────────────────────────────────────────────
#  Balanced-ternary “motif” Shor front-end   (notebook friendly)
#  --------------------------------------------------------------
#  Call   factor_motif(N)   where N is an *odd composite* < 2**64
#  It returns (p, q, attempts, base_a, order_r)
# ────────────────────────────────────────────────────────────────

import math, random
from typing import List, Tuple

# 1.  Balanced-ternary helpers  ─────────────────────────────────
def to_balanced_ternary(x: int) -> List[int]:
    """Little-endian list of trits in {-1,0,1}."""
    if x == 0:
        return [0]
    trits = []
    while x:
        x, r = divmod(x, 3)
        if r == 2:          # use −1 with carry-over
            x += 1
            r = -1
        trits.append(r)
    return trits

def from_balanced_ternary(trits: List[int]) -> int:
    return sum(t * 3**i for i, t in enumerate(trits))

def mul_trits_mod_N(trits: List[int], a: int, N: int) -> List[int]:
    """Multiply the exponent (stored in balanced ternary) by a (mod N)."""
    exp_val = (from_balanced_ternary(trits) * a) % N
    return to_balanced_ternary(exp_val)

# 2.  Order-finding loop  ───────────────────────────────────────
def find_order(a: int, N: int, limit: int | None = None) -> int:
    """
    Return the smallest r > 0 with  a**r ≡ 1 (mod N)
    by iterating the balanced-ternary motif update.
    """
    if limit is None:
        limit = 6 * N                  # generous guard
    trits = [1]                        # exponent = 1
    for r in range(1, limit + 1):
        trits = mul_trits_mod_N(trits, a, N)
        if len(trits) == 1 and trits[0] == 1:
            return r                   # order found
    raise RuntimeError("order loop exceeded limit")

# 3.  Shor post-processing  ─────────────────────────────────────
def factor_motif(N: int,
                 *,
                 rng_seed: int | None = None
                ) -> Tuple[int, int, int, int, int]:
    """
    Factor an *odd composite* N using the motif order-finder.
    Returns (p, q, attempts, base_a, order_r).
    """
    if rng_seed is not None:
        random.seed(rng_seed)

    attempts = 0
    while True:
        attempts += 1
        a = random.randrange(2, N - 1)
        if math.gcd(a, N) != 1:     # need a coprime base
            continue
        r = find_order(a, N)
        if r % 2:                   # order must be even
            continue
        x = pow(a, r // 2, N)
        if x in (1, N - 1):         # trivial root
            continue
        p = math.gcd(x - 1, N)
        q = math.gcd(x + 1, N)
        if p * q == N and p not in (1, N) and q not in (1, N):
            return p, q, attempts, a, r

# ────────────────────────────────────────────────────────────────
#  Example usage  (uncomment to run)
# ────────────────────────────────────────────────────────────────
# N_demo = 355207          # 599 × 593 (comment only)
# p, q, tries, a, r = factor_motif(N_demo, rng_seed=42)
# print(f"Found factors: {p} × {q}  (base {a}, order {r}, attempts {tries})")
